In [1]:
#!M
# !pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1
# %pip install --upgrade pip
# %pip install -r /home/jupyter/work/resources/riiidNew/requirements.txt --upgrade
# %pip install scipy  --ignore-installed scipy --upgrade
# %pip install pyarrow==0.17.1
# %pip install bayesian-optimization
print("ok")

ok


In [2]:
#!L
# if str(Path.home()) == "/root":  # DATASPHERE
import sys
import gc
from pathlib import Path
import numpy as np
import pandas as pd
import pickle
# import pyarrow
from collections import defaultdict
from catboost import CatBoostClassifier, Pool

sys.path.append(str(Path.cwd()/'riiidNew'))
from preprocess import preprocess_train_data, divide_nan
from catboost_bayesian_search import bayesian_catboost_search, bayesian_catboost_searchCV, param_adjust_dtypes
import config
print(Path.cwd())

/home/riiid/riiidNew


In [3]:
# Error handling, ignore all
np.seterr(divide = 'ignore', invalid = 'ignore')


if str(Path.home()) == "/home/sergey":   # Home computer
    data_path = Path("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata")
    kaggle_path = Path.cwd()/"kaggle_tmp"
    questions_df = pd.read_csv(data_path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    # test_df = pd.read_feather(data_path/"cv1_valid_1e4.feather", columns=config.dtypes.keys())
    train_df = pd.read_feather(data_path/"train_1e5.feather", columns=config.dtypes.keys())
    # test_df = pd.read_feather(data_path/"cv1_valid_1e4.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(data_path/"cv1_train.feather", columns=config.dtypes.keys())
    # test_df = pd.read_feather(data_path/"cv1_valid.feather", columns=config.dtypes.keys())

elif str(Path.home()) == "/root":   # DATASPHERE

    data_path = Path.cwd()/"riiidNew"/"data1"
    kaggle_path = Path.cwd()/"riiidNew"/"kaggle"
    questions_df = pd.read_csv(data_path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    train_df = pd.read_feather(data_path/"train.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(data_path/"train_5e6.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(data_path/"train_1e5.feather", columns=config.dtypes.keys())

elif str(Path.home()) == "/home/riiid":   # Virtual machine ycloud

    data_path = Path.cwd()/"data"
    kaggle_path = Path.cwd()/"kaggle"
    questions_df = pd.read_csv(data_path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    # train_df = pd.read_feather(data_path/"train.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(data_path/"train_5e6.feather", columns=config.dtypes.keys())
    # test_df = pd.read_feather(data_path/"cv2_test.feather", columns=config.dtypes.keys())
    train_df = pd.read_feather(data_path/"cv2_train_1e5.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(data_path/"train_1e5.feather", columns=config.dtypes.keys())
#     train_df = train_df.iloc[:100]

print(f"train_df shape = {train_df.shape}")

train_df shape = (100000, 10)


In [4]:
train_df.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,user_answer
0,32933156,0,705741139,128,0,0,1,NaN,<NA>,0
1,32933157,20666,705741139,7860,0,1,1,16000.0,False,0
2,32933158,39172,705741139,7922,0,2,1,19000.0,False,1
3,32933159,58207,705741139,156,0,3,1,17000.0,False,2
4,32933160,75779,705741139,51,0,4,1,17000.0,False,0


In [5]:
# test_df.head()

In [6]:
print(config.prior_params)

{'loss_function': 'Logloss', 'eval_metric': 'AUC', 'custom_metric': 'AUC:hints=skip_train~false', 'task_type': 'GPU', 'grow_policy': 'Lossguide', 'iterations': 2, 'thread_count': -1, 'learning_rate': 0.1, 'random_seed': 0, 'bootstrap_type': 'Bayesian', 'l2_leaf_reg': 0.1, 'depth': 10, 'max_leaves': 10, 'border_count': 128, 'verbose': 250, 'od_type': 'Iter', 'od_wait': 50}


In [7]:
horizon = 26
model_name = "model40"

In [8]:
# Obtain new train_df and validation test_df
# train_df = pd.read_feather(path/"cv2_train.feather", columns=config.dtypes.keys())
# train_df = pd.read_feather(data_path/"train.feather", columns=config.dtypes.keys())


# Preprocess for model fitting
train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
# test_user_agg, test_content_agg, test_df = preprocess_train_data(test_df, questions_df, config.target, config.dtypes)
print("Preprocess for model fitting ok")

print("train_df.columns")
print(train_df.columns)
# print("test_df.columns")
# print(test_df.columns)

#test_df = train_df.groupby('user_id').tail(horizon)
test_df = train_df.groupby('user_id').sample(horizon)
train_df.drop(test_df.index, inplace = True)

# Training and validating data
train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
print("train_set Pool ready")

val_set = Pool(test_df[config.features], label = test_df[config.target], cat_features=config.cat_features)

print(" val_set Pool ready")
# val_set = Pool(valid_df[features], label = valid_df[target])
# params = param_adjust_dtypes(config.prior_params,
#                              config.pds_dtypes,
#                              optimizer_max["params"])
params = config.prior_params
# params["learning_rate"] = 3e-2
# params['iterations'] = 25000

print("params")
print(params)

model = CatBoostClassifier(**params)

# Fitting
model.fit(train_set, eval_set = val_set, use_best_model = True)
print(f"kaggle_path {kaggle_path}")

model.save_model(f"{kaggle_path}/catboost.{model_name}")
print("model.get_best_score()")
print(model.get_best_score())

Preprocess for model fitting ok
train_df.columns
Index(['row_id', 'timestamp', 'user_id', 'content_id', 'content_type_id',
       'task_container_id', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation',
       'user_answer', 'user_correctness', 'part', 'content_count',
       'question_difficulty'],
      dtype='object')


AttributeError: 'DataFrameGroupBy' object has no attribute 'sample'

In [ ]:
# {'learn': {'Logloss': 0.5831425763098211, 'AUC': 0.7542276382446289}, 'validation': {'Logloss': 0.599964516913287, 'AUC': 0.7367952764034271}}

In [ ]:
from pathlib import Path
Path.home()

# Inference

In [ ]:
user_sum_dict =  train_user_agg['sum'].astype('int32').to_dict(defaultdict(int))
user_count_dict =  train_user_agg['count'].astype('int32').to_dict(defaultdict(int))
content_sum_dict =  train_content_agg['sum'].astype('int32').to_dict(defaultdict(int))
content_count_dict =  train_content_agg['count'].astype('int32').to_dict(defaultdict(int))

for filename, dic in zip(["user_sum_dict", "user_count_dict", "content_sum_dict", "content_count_dict"],
                         [user_sum_dict, user_count_dict, content_sum_dict, content_count_dict]):
# for filename, dic in dict_.items():
    with open(f'{kaggle_path}/{filename}.{model_name}.pickle', 'wb') as handle:
        pickle.dump(dic, handle)

In [ ]:
train_df.head()
del train_df
gc.collect()

In [ ]:
%%time
validaten_flg = True
if validaten_flg:
    from emulator import Iter_Valid
    iter_test = Iter_Valid(test_df,max_user=1000)
    predicted = []
    def set_predict(df):
        predicted.append(df)
else:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict


In [ ]:
%%time
prior_test_df = None
for (test_df, sample_prediction_df) in iter_test:
    if prior_test_df is not None:
        prior_test_df[config.target] = eval(test_df['prior_group_answers_correct'].iloc[0])

        # Exclude lectures
        prior_test_df = prior_test_df[prior_test_df['content_type_id'] == 0].reset_index(drop = True)

        prior_test_np = prior_test_df[["user_id", "content_id", config.target]].values
        for user_id, content_id, answered_correctly in prior_test_np:
            user_sum_dict[user_id] += answered_correctly
            user_count_dict[user_id] += 1
            content_sum_dict[content_id] += answered_correctly
            content_count_dict[content_id] += 1

    prior_test_df = test_df.copy()
    
    # Exclude lectures
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop = True)

    test_df.drop(labels="part", axis=1, inplace=True)
    test_df.content_id = test_df.content_id.astype(int)
    
    test_df = pd.concat(
        [test_df.reset_index(drop=True),
         questions_df.reindex(test_df['content_id'].values).reset_index(drop=True)],
         axis=1, sort=False, join="outer")

    # Fill NaN values in the 'prior_question_had_explanation' columns
    test_df['prior_question_had_explanation'] = \
        test_df['prior_question_had_explanation'].fillna(False).astype('bool')


    # test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
    # user_sum = np.zeros(len(test_df), dtype = np.int32)
    # user_count = np.zeros(len(test_df), dtype = np.int32)
    # content_sum = np.zeros(len(test_df), dtype = np.int32)
    # content_count = np.zeros(len(test_df), dtype = np.int32)

    for col in config.new_features:
        test_df[col] = 0

    user_correctness_offset = test_df.columns.get_loc('user_correctness')
    content_count_offset = test_df.columns.get_loc('content_count')
    question_difficulty_offset = test_df.columns.get_loc('question_difficulty')


    # Error! for i, (user_id, content_id) in prior_test_df[["user_id", "content_id"]].iterrows():
    # prior_test_np = prior_test_df[["user_id", "content_id"]].values
    # for i, (user_id, content_id) in enumerate(prior_test_np):
    for i, (user_id, content_id) in enumerate(zip(test_df['user_id'].values, test_df['content_id'].values)):
        # user_correctness
        # user_correctness = np.nan if user_count_dict[user_id] == 0 else user_sum_dict[user_id] / user_count_dict[user_id]
        test_df.iloc[i, user_correctness_offset] = divide_nan(user_sum_dict[user_id], user_count_dict[user_id])

        # content_count
        test_df.iloc[i, content_count_offset] = int(content_count_dict[content_id])

        # content_id
#         content_id_val = np.nan if content_count_dict[content_id] == 0 else content_sum_dict[content_id] / content_count_dict[content_id]
        test_df.iloc[i, question_difficulty_offset] = divide_nan(content_sum_dict[content_id], content_count_dict[content_id])

        # user_sum[i] = user_sum_dict[user_id]
        # user_count[i] = user_count_dict[user_id]
        # content_sum[i] = content_sum_dict[content_id]
        # content_count[i] = content_count_dict[content_id]

    # test_df['user_correctness'] = user_sum / user_count
    # test_df['content_count'] = content_count.astype(int)
    # test_df['question_difficulty'] = content_sum / content_count

    test_df.prior_question_elapsed_time = test_df.prior_question_elapsed_time // 2000

    test_df[config.target] = model.predict_proba(test_df[config.features])[:,1]
    set_predict(test_df[['row_id', config.target]])
